In [40]:
from graphdatascience import GraphDataScience
from getpass import getpass

In [41]:
neo4j_password = getpass("Neo4j password:")

In [42]:
neo4j_url = "neo4j+s://5f8297f1.databases.neo4j.io"
auth = ("neo4j", neo4j_password)
gds = GraphDataScience(neo4j_url, auth=auth)

## Project the whole graph (50M profiles X 4 identifiers)
We expect many distinct profiles to share the same birthdate, so don't include DOB in this projection

In [43]:
g_identity, result = gds.graph.project("g_identity",
                                       ["UserDescription", "PhoneNumber", "Email", "SocialSecurityNumber", "PersonName"],
                                       {"HAS_PHONE":{"orientation":"UNDIRECTED"}, 
                                        "HAS_EMAIL":{"orientation":"UNDIRECTED"}, 
                                        "HAS_SOCIAL_SECURITY_NUMBER":{"orientation":"UNDIRECTED"}, 
                                        "HAS_NAME":{"orientation":"UNDIRECTED"}},
                                       readConcurrency=20)

Loading:   0%|          | 0/100 [00:00<?, ?%/s]

In [44]:
result

nodeProjection            {'SocialSecurityNumber': {'label': 'SocialSecu...
relationshipProjection    {'HAS_NAME': {'aggregation': 'DEFAULT', 'orien...
graphName                                                        g_identity
nodeCount                                                         248520095
relationshipCount                                                 400016000
projectMillis                                                         75551
Name: 0, dtype: object

## Run Weakly Connected Components in stats mode
Make sure the component distribution makes sense.

In [45]:
wcc_stats = gds.wcc.stats(g_identity,
                          concurrency=20)

WCC:   0%|          | 0/100 [00:00<?, ?%/s]

In [46]:
wcc_stats['componentDistribution']

{'min': 5,
 'p5': 5,
 'max': 25,
 'p999': 9,
 'p99': 9,
 'p1': 5,
 'p10': 5,
 'p90': 5,
 'p50': 5,
 'p25': 5,
 'p75': 5,
 'p95': 5,
 'mean': 5.122458072360732}

In [47]:
wcc_stats['componentCount']

48515789

In [48]:
gds.wcc.write(g_identity,
               writeProperty="wccId",
               minComponentSize=6,
               concurrency=20)

WCC:   0%|          | 0/100 [00:00<?, ?%/s]

writeMillis                                                          33479
nodePropertiesWritten                                             13136655
componentCount                                                    48515789
componentDistribution    {'min': 5, 'p5': 5, 'max': 25, 'p999': 9, 'p99...
postProcessingMillis                                                  1579
preProcessingMillis                                                      0
computeMillis                                                        13538
configuration            {'writeProperty': 'wccId', 'jobId': '47202dc3-...
Name: 0, dtype: object

## Drop the graph projection to free up memory

In [49]:
gds.graph.drop(g_identity)

graphName                                                       g_identity
database                                                             neo4j
databaseLocation                                                     local
memoryUsage                                                               
sizeInBytes                                                             -1
nodeCount                                                        248520095
relationshipCount                                                400016000
configuration            {'relationshipProjection': {'HAS_NAME': {'aggr...
density                                                                0.0
creationTime                           2024-06-10T12:16:19.479525968+00:00
modificationTime                       2024-06-10T12:17:35.041569086+00:00
schema                   {'graphProperties': {}, 'nodes': {'SocialSecur...
schemaWithOrientation    {'graphProperties': {}, 'nodes': {'SocialSecur...
Name: 0, dtype: object

## The biggest component has 25 nodes. It's reasonable to calculate similarity with Cypher.

In [50]:
matching_result_df = gds.run_cypher("""
    CALL apoc.periodic.iterate(
        "MATCH (u:UserDescription) WHERE u.wccId IS NOT NULL
        WITH u.wccId AS wccId, collect(u) AS component
        RETURN component",
        'WITH apoc.coll.combinations(component, 2, 2) AS pairs
        UNWIND pairs AS pair
        WITH pair[0] AS u1, pair[1] AS u2
        CALL {
            WITH u1, u2
            MATCH (u1)-[:HAS_DOB]->(n1), (u2)-[:HAS_DOB]->(n2)
            WHERE apoc.text.levenshteinDistance(n1.birthdateString, n2.birthdateString) <= 2
            RETURN DISTINCT {property: "dob", matchType: CASE WHEN n1.birthdateString = n2.birthdateString THEN "Exact" ELSE "Fuzzy" END} AS matched
            
            UNION ALL

            WITH u1, u2
            MATCH (u1)-[:HAS_EMAIL]->(n1), (u2)-[:HAS_EMAIL]->(n2)
            WHERE apoc.text.levenshteinDistance(n1.email, n2.email) <= 2
            RETURN DISTINCT {property: "email", matchType: CASE WHEN n1.email = n2.email THEN "Exact" ELSE "Fuzzy" END} AS matched
            
            UNION ALL

            WITH u1, u2
            MATCH (u1)-[:HAS_NAME]->(n1), (u2)-[:HAS_NAME]->(n2)
            WHERE apoc.text.levenshteinDistance(n1.fullName, n2.fullName) <= 2
            RETURN DISTINCT {property: "name", matchType: CASE WHEN n1.fullName = n2.fullName THEN "Exact" ELSE "Fuzzy" END} AS matched
                    
            UNION ALL

            WITH u1, u2
            MATCH (u1)-[:HAS_PHONE]->(n1), (u2)-[:HAS_PHONE]->(n2)
            WHERE apoc.text.levenshteinDistance(n1.phoneNumber, n2.phoneNumber) <= 2
            RETURN DISTINCT {property: "phone", matchType: CASE WHEN n1.phoneNumber = n2.phoneNumber THEN "Exact" ELSE "Fuzzy" END} AS matched

            UNION ALL
            
            WITH u1, u2
            MATCH (u1)-[:HAS_SOCIAL_SECURITY_NUMBER]->(n1), (u2)-[:HAS_SOCIAL_SECURITY_NUMBER]->(n2)
            WHERE apoc.text.levenshteinDistance(n1.socialSecurityNumber, n2.socialSecurityNumber) <= 2
            RETURN DISTINCT {property: "ssn", matchType: CASE WHEN n1.socialSecurityNumber = n2.socialSecurityNumber THEN "Exact" ELSE "Fuzzy" END} AS matched
        }
        WITH u1, u2, collect(matched) AS cluesMatched
        WHERE size(cluesMatched) > 1
        
        MERGE (u1)-[s:IS_SIMILAR]->(u2)
        SET s.exactMatchedProperties = [clue IN cluesMatched WHERE clue["matchType"] = "Exact" | clue["property"]],
        s.fuzzyMatchedProperties = [clue IN cluesMatched WHERE clue["matchType"] = "Fuzzy" | clue["property"]]',
    {parallel:True})
    """)

In [51]:
matching_result_df

,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
0,144,1439101,150,1439101,0,0,0,{},"{'total': 144, 'errors': {}, 'committed': 144,...","{'total': 1439101, 'errors': {}, 'committed': ...",False,{},"{'relationshipsDeleted': 0, 'relationshipsCrea..."


In [52]:
sim_result = gds.run_cypher("""
    MATCH (u1)-[s:IS_SIMILAR]->(u2)
    RETURN u1.userId As u1Id, u2.userId AS u2Id, s.exactMatchedProperties AS exactMatchedProperties, s.fuzzyMatchedProperties AS fuzzyMatchedProperties
""")

In [53]:
sim_result.head()

,u1Id,u2Id,exactMatchedProperties,fuzzyMatchedProperties
0,kvmjhefi-bxb5-hzis-viny-h3p8anfgzcj5,t07u2h0p-6ej3-06zg-ddko-vf6qrtrz1z7t,[ssn],[dob]
1,zs8y17qo-l31o-s63u-p678-hyjrqr0k4i9e,y9phlm86-upec-j5te-p3vk-xtptoueghmpu,[ssn],[dob]
2,5lhy9ro6-y51k-e9wj-08ln-t4odkn4wq9pq,fh8fgtl8-96zs-6cxx-8tbd-u1dt4bwge4ie,[ssn],[dob]
3,3jcnqak1-9w51-500m-fthh-0vz8rzlgzpy6,4mnqfs4y-weo4-i9fr-klh9-nrf00a0ryuj8,[ssn],[dob]
4,ebjpqce5-06tt-bkla-4jjz-lbkmdpxree64,da3cbe42-c10d-40ed-ac14-37b3481ee31c,"[dob, email, name, phone]",[ssn]


In [54]:
sim_result.shape

(24746, 4)

In [55]:
sim_result['exact_matched'] = sim_result['exactMatchedProperties'].str.join(", ")
sim_result['fuzzy_matched'] = sim_result['fuzzyMatchedProperties'].str.join(", ")

In [56]:
sim_result.groupby(['exact_matched', 'fuzzy_matched']).count()

,,u1Id,u2Id,exactMatchedProperties,fuzzyMatchedProperties
exact_matched,fuzzy_matched,,,,
,"dob, ssn",2,2,2,2
"dob, email",,3,3,3,3
"dob, email, name","phone, ssn",415,415,415,415
"dob, email, name, phone",ssn,486,486,486,486
"dob, email, name, phone, ssn",,109,109,109,109
"dob, email, name, ssn",phone,490,490,490,490
"dob, name, phone, ssn",email,500,500,500,500
"dob, phone",,1,1,1,1
"dob, ssn",,50,50,50,50


## Remove the wccId property from UserDescription nodes so it doesn't confuse future runs of the matching job.

In [57]:
gds.run_cypher("""
    CALL apoc.periodic.iterate(
        "MATCH (u:UserDescription) WHERE u.wccId IS NOT NULL
        RETURN u",
        "SET u.wccId = NULL",
        {parallel:True}
    )
""")

,batches,total,timeTaken,committedOperations,failedOperations,failedBatches,retries,errorMessages,batch,operations,wasTerminated,failedParams,updateStatistics
0,293,2925312,6,2925312,0,0,0,{},"{'total': 293, 'errors': {}, 'committed': 293,...","{'total': 2925312, 'errors': {}, 'committed': ...",False,{},"{'relationshipsDeleted': 0, 'relationshipsCrea..."
